# Velocity Control Kinematic Simulation

### Requirements
Pip
- numpy
- scipy
- matplotlib

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from habitat.utils.controller_wrapper import ContinuousController

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  

In [ ]:
# ----- HYPERPARAMS
# Sim
SPEEDUP = 2

TIME_HORIZON = 5
SIM_HZ = 120
CONTROL_HZ = 10
VIZ_HZ = 24 / SPEEDUP

# Viz
R_ROBOT = 0.2
BUTT_SAMPLES = 20
X_LIM = [-2, 2]
Y_LIM = [-2, 2]

In [ ]:
# ----- SIMULATION ENV
class Env:
    def __init__(self, hz, pos_init=None):
        self.pos_state = np.zeros(3) if pos_init is None else np.array(pos_init)
        self.vel_state = np.zeros(2)
        self.dt = 1.0 / hz
        
    def get_pose(self):
        return self.pos_state
    
    def step(self, vel_input=None):
        if vel_input is not None:
            self.vel_state = vel_input
            
        self.pos_state[0] += self.vel_state[0] * np.cos(self.pos_state[2]) * self.dt
        self.pos_state[1] += self.vel_state[0] * np.sin(self.pos_state[2]) * self.dt
        self.pos_state[2] += self.vel_state[1] * self.dt

In [ ]:
# ----- VISUALIZER
class Viz:
    def __init__(self):
        self.frames = []
        
        # Robot local vertices
        xy_ul = np.array([R_ROBOT / 2, R_ROBOT])
        xy_ur = np.array([R_ROBOT / 2, -R_ROBOT])
        xy_l = np.array([-R_ROBOT / 2, R_ROBOT])
        xy_r = np.array([-R_ROBOT / 2, -R_ROBOT])
        
        xy_list = [xy_r, xy_ur, xy_ul, xy_l]
        for phi in np.linspace(np.pi / 2, 3 * np.pi / 2, BUTT_SAMPLES):
            xy_list.append(np.array([R_ROBOT * (np.cos(phi) - 0.5), R_ROBOT * np.sin(phi)]))
            
        self.xy_verts_local = np.array(xy_list)
        
    def _draw_robot(self, xyt, color):
        theta = xyt[2]
        ct = np.cos(theta)
        st = np.sin(theta)
        trans = np.array([[ct, -st], [st, ct]])
        
        xy_verts_transformed = (trans @ self.xy_verts_local.T).T + xyt[None, :2]
        
        self.ax.plot(xyt[0], xyt[1], color=color, marker='o')
        self.ax.plot(xy_verts_transformed[:, 0], xy_verts_transformed[:, 1], color=color)
        
    def _render_frame(self, frame):
        plt.cla()
        
        self.ax.set_xlim(*X_LIM)
        self.ax.set_ylim(*Y_LIM)
        self.ax.set_aspect("equal")
        
        self._draw_robot(frame[1], '0.7')
        self._draw_robot(frame[0], '0')
        
    def clear_frames(self):
        self.frames = []
    
    def record_frame(self, xyt_robot, xyt_goal):
        self.frames.append((xyt_robot, xyt_goal))
        
    def animate(self, playback_speed=1.0):  
        plt.ioff()
        fig, self.ax = plt.subplots()
        
        return animation.FuncAnimation(fig, self._render_frame, frames=self.frames, interval=1000 / VIZ_HZ / playback_speed, repeat=False)
    
    
    

In [ ]:
# ----- CONTROLLER
from omegaconf import OmegaConf

def create_controller(
    v_max=0.3,
    w_max=0.45,
    acc_lin=0.2,
    acc_ang=0.6,
    max_heading_ang=np.pi / 6,
    lin_error_tol=0.01,
    ang_error_tol=0.025,
    track_yaw=True,
):
    # Generate config
    cfg_dict = {
        "v_max": v_max,
        "w_max": w_max,
        "acc_lin": acc_lin,
        "acc_ang": acc_ang,
        "lin_error_tol": lin_error_tol,
        "ang_error_tol": ang_error_tol,
        "max_heading_ang": max_heading_ang,
    }
    cfg = OmegaConf.create(cfg_dict)

    # Instatiate controller
    return ContinuousController(cfg)

In [ ]:
# ----- GOAL
GOAL = [0.25, 0, 0]  # move_forward
#GOAL = [0, 0, np.deg2rad(30)]  # turn_left
#GOAL = [0, 0, np.deg2rad(30)]  # turn_right

In [ ]:
# ----- MAIN
# initialize objects
xyt_init = np.array([0.5, 0.5, 1.0]) * np.random.randn(3)
xyt_goal = np.array(GOAL)

viz = Viz()
env = Env(SIM_HZ, xyt_init)
w2v_controller = create_controller()

# set goal
w2v_controller.set_goal(xyt_goal, start=xyt_init, relative=True)
xyt_goal_global = w2v_controller.controller.xyt_goal  # get global goal state from controller

# simulate
t_control = 0
t_viz = 0
dt_control = 1 / CONTROL_HZ
dt_viz = 1 / VIZ_HZ
for t in np.linspace(0, TIME_HORIZON, int(TIME_HORIZON * SIM_HZ)):
    xyt = env.get_pose()
    
    vel_command = None
    if t >= t_control:
        vel_command = w2v_controller.forward(xyt)
        t_control += dt_control
    if t >= t_viz:
        viz.record_frame(xyt.copy(), xyt_goal_global.copy())
        t_viz += dt_viz
    
    env.step(vel_command)

In [ ]:
# animate
viz.animate(playback_speed=SPEEDUP)